In [1]:
#Common imports
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
import cvxpy as cp
from numpy import linalg
from sklearn.datasets import make_moons, make_circles, make_classification
import mosek



# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
import _scs_gpu

In [2]:
cp.installed_solvers()

['CVXOPT', 'ECOS', 'GLPK', 'GLPK_MI', 'MOSEK', 'OSQP', 'SCS']

In [3]:
list(range(1,202))

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

In [4]:
df_train = pd.read_csv("train.csv", names=list(range(1,202)))
df_test = pd.read_csv("test.csv",names=list(range(1,202)))

In [5]:
df_train.head()

,1,2,3,4,5,6,7,8,9,10,...,192,193,194,195,196,197,198,199,200,201
0,0.0,-0.36,-0.91,-0.99,-0.57,-1.38,-1.54,-1.64,1.29,0.65,...,1.15,-0.05,-0.09,0.02,1.75,1.58,0.12,0.30,2.44,-1.26
1,1.0,-1.40,-1.90,0.09,0.29,-0.30,-1.30,1.13,-2.38,-1.16,...,0.48,0.24,-0.16,-0.48,-0.02,-0.35,-0.27,-0.20,-0.92,-0.46
2,1.0,-0.43,1.45,-0.68,-1.58,0.32,-0.14,0.23,-1.01,-0.39,...,-0.94,0.11,-1.30,-0.24,0.74,0.88,1.37,0.12,0.01,-0.56
3,1.0,-0.76,0.30,-0.57,-0.33,-1.50,1.84,1.37,0.23,0.66,...,-0.42,0.06,-1.05,0.35,-0.24,-0.69,1.31,-0.18,-1.54,-1.70
4,0.0,-0.76,1.36,0.00,-1.44,-1.27,-0.76,-1.42,-0.58,0.11,...,-0.44,1.45,-1.18,-1.13,-0.14,0.04,0.33,1.20,-0.81,-1.16


In [6]:
df_train.describe()

,1,2,3,4,5,6,7,8,9,10,...,192,193,194,195,196,197,198,199,200,201
count,8500.000000,8500.000000,8500.000000,8500.000000,8500.000000,8500.000000,8500.000000,8500.000000,8500.000000,8500.000000,...,8500.000000,8500.000000,8500.000000,8500.000000,8500.000000,8500.000000,8500.000000,8500.000000,8500.000000,8500.000000
mean,0.500118,0.015781,-0.013220,0.000401,0.007042,0.014908,0.007422,0.009867,-0.005094,-0.003275,...,0.002894,-0.005624,0.004547,0.008327,0.025432,0.004219,0.006253,-0.010878,0.002534,0.005359
std,0.500029,1.053792,1.057382,1.040262,1.050884,1.057882,1.039831,1.047432,1.048173,1.058878,...,1.049972,1.051867,1.064075,1.042985,1.047585,1.041096,1.062399,1.036885,1.040154,1.051150
min,0.000000,-3.980000,-4.720000,-3.570000,-4.160000,-4.230000,-4.600000,-3.560000,-3.870000,-3.720000,...,-4.140000,-4.340000,-4.040000,-3.820000,-3.480000,-3.930000,-5.120000,-3.620000,-4.080000,-4.260000
25%,0.000000,-0.680000,-0.730000,-0.692500,-0.710000,-0.690000,-0.690000,-0.710000,-0.710000,-0.720000,...,-0.710000,-0.720000,-0.710000,-0.700000,-0.690000,-0.690000,-0.710000,-0.720000,-0.700000,-0.690000
50%,1.000000,0.020000,-0.010000,0.010000,0.020000,0.030000,0.000000,0.010000,-0.010000,0.000000,...,-0.010000,-0.020000,0.000000,0.010000,0.030000,0.020000,0.005000,0.000000,0.010000,0.020000
75%,1.000000,0.730000,0.710000,0.720000,0.720000,0.732500,0.730000,0.720000,0.720000,0.690000,...,0.702500,0.700000,0.740000,0.710000,0.720000,0.720000,0.720000,0.690000,0.710000,0.700000
max,1.000000,4.280000,3.780000,3.920000,3.770000,3.880000,3.950000,3.500000,4.020000,3.980000,...,4.100000,4.180000,3.800000,4.250000,3.570000,3.750000,4.210000,3.570000,4.110000,3.930000


In [7]:

scaler = MinMaxScaler()



x_train = df_train.drop(1, axis = 1)
y_train = df_train[1].to_numpy()
y_train = np.where(y_train == 0, -1, 1)


x_train = scaler.fit_transform(x_train)

x_test = df_test.drop(1, axis = 1)
y_test = df_test[1].to_numpy()
y_test = np.where(y_test == 0, -1, 1)


x_test = scaler.transform(x_test)

In [8]:
x_train

array([[0.43825666, 0.44823529, 0.34445928, ..., 0.54520167, 0.7960928 ,
        0.36630037],
       [0.31234867, 0.33176471, 0.48865154, ..., 0.47566064, 0.38583639,
        0.46398046],
       [0.42978208, 0.72588235, 0.3858478 , ..., 0.5201669 , 0.4993895 ,
        0.45177045],
       ...,
       [0.62469734, 0.64705882, 0.47396529, ..., 0.41585535, 0.53357753,
        0.4029304 ],
       [0.41525424, 0.69176471, 0.40186916, ..., 0.91655076, 0.17338217,
        0.28815629],
       [0.57869249, 0.44117647, 0.45660881, ..., 0.2837274 , 0.54090354,
        0.64957265]])

In [9]:
#len(x_train)
x_train.shape[0]

8500

In [10]:
def svm_train_dual_soft (data_train , label_train , regularisation_para_C):
    n_samples = len(data_train)
    n = data_train.shape[1]
    d = data_train.shape[0]

    alpha = cp.Variable(shape=(d) , pos=True)
    C = cp.Parameter()
    C.value = regularisation_para_C
    
    H = np.dot((label_train[:,None] * data_train) , (  label_train[:,None] * data_train).T)

   

    obj = cp.Maximize(cp.sum(alpha)-(1/2)*cp.quad_form(alpha,H))

    constraint_1 = [alpha >= 0]
    constaint_2 = [alpha <= C/n]
    constraint_3 = [(label_train@alpha) == 0]

    constraint = constraint_1 + constaint_2 + constraint_3


    prob = cp.Problem(obj, constraint)

    results = prob.solve(solver=cp.MOSEK)
    print (prob.status)
    #print(alpha.value)
    
    
    aaa = alpha.value
    aaa = np.where(aaa>1e-5 , aaa, 0)
    w_dual = ((label_train.T * aaa.T) @ data_train)
    aaa = np.where(aaa>(regularisation_para_C/n)-0.01 , 0, aaa)
    S = (aaa > 0).flatten()
    b = label_train[S] - np.dot(data_train[S],w_dual)
    
    weight_w_bias = np.concatenate(([b[0]],w_dual))
    
    return weight_w_bias
    

    

    
    
def svm_predict_dual(data_test , label_test , svm_model):
    
    predicted = []
        
    for x1 in data_test:
            
            results = np.where((np.dot(svm_model[1:], x1) + svm_model[0]) >= 0.0 , 1, -1)
            predicted.append(results)
    
    
    return accuracy_score(label_test, predicted)

In [11]:
def svm_train_dual_hard (data_train , label_train):
    n_samples = len(data_train)
    n = data_train.shape[1]
    d = data_train.shape[0]

    alpha = cp.Variable(shape=(d),pos=True)
#     C = cp.Parameter()
#     C.value = regularisation_para_C
    
    H = np.dot((label_train[:,None] * data_train) , (  label_train[:,None] * data_train).T)

   

    obj = cp.Maximize(cp.sum(alpha)-(1/2)*cp.quad_form(alpha,H))

    constraint_1 = [alpha >= 0]
    #constaint_2 = [alpha <= C]
    constraint_3 = [(label_train@alpha) == 0]

#     constraint = constraint_1 + constaint_2 + constraint_3
    constraint = constraint_1 + constraint_3


    prob = cp.Problem(obj, constraint)

    results = prob.solve(solver=cp.MOSEK)
    print (prob.status)
    #print(alpha.value)
    
    
    aaa = alpha.value
    aaa = np.where(aaa>1e-5 , aaa, 0)
    w_dual = ((label_train.T * aaa.T) @ data_train)
    #aaa = np.where(aaa>regularisation_para_C-0.1 , 0, aaa)
    S = (aaa > 0).flatten()
    b = label_train[S] - np.dot(data_train[S],w_dual)
    
    weight_w_bias = np.concatenate(([b[0]],w_dual))
    
    return weight_w_bias
    

    

    
    
def svm_predict_dual(data_test , label_test , svm_model):
    
    predicted = []
        
    for x1 in data_test:
            
            results = np.where((np.dot(svm_model[1:], x1) + svm_model[0]) >= 0.0 , 1, -1)
            predicted.append(results)
    
    
    return accuracy_score(label_test, predicted)

In [12]:
def svm_train_primal_hard (data_train, label_train):
    n_samples = len(data_train)
    d = data_train.shape[1]
    n = data_train.shape[0]

    W = cp.Variable((d))
    bias = cp.Variable()

    obj = cp.Minimize(1/2*cp.norm(W,2))


    constranit_1 = [cp.multiply(label_train,(data_train@W+bias)) >=1]


    constraints = constranit_1 

    prob = cp.Problem(obj, constraints)

    prob.solve(solver=cp.MOSEK)

    print (prob.status)

    w = W.value
    b = bias.value
    
    weight_w_bias = np.concatenate(([b],w))
    
    return weight_w_bias


In [13]:
def svm_train_primal_soft (data_train, label_train, regularisation_para_C):
    n_samples = len(data_train)
    n = data_train.shape[0]
    d = data_train.shape[1]

    W = cp.Variable((d))
    bias = cp.Variable()
    epi = cp.Variable(n_samples)
    C = cp.Parameter()
    C.value = regularisation_para_C
    a = cp.Constant(n_samples)
    obj = cp.Minimize(1/2*cp.norm(W,2) + (C/n_samples)*cp.sum(epi))

    constranit_1 = [cp.multiply(label_train,(data_train@W+bias)) >= 1-epi]

    constraint_2 = [epi >= 0]

    constraints = constranit_1 + constraint_2

    prob = cp.Problem(obj, constraints)

    prob.solve(solver=cp.MOSEK)

    print (prob.status)

    w = W.value
    b = bias.value
    ee = epi.value
    
    weight_w_bias = np.concatenate(([b],w))
    
    return weight_w_bias

    

def svm_predict_primal(data_test , label_test , svm_model):
    
    
    predicted = []
        
    for x1 in data_test:
            
            results = np.where((np.dot(svm_model[1:], x1) + svm_model[0]) >= 0.0 , 1, -1)
            predicted.append(results)
    
    
    return accuracy_score(label_test, predicted)
    

In [14]:
X_class, y_class = make_classification(n_features=2, n_redundant=0, n_informative=2,
                           random_state=1, n_clusters_per_class=1)
y_class = np.where(y_class == 0, -1, 1)


X_circles, y_circles = make_circles(n_samples=100, noise=0.2, factor=0.5, random_state=1)

y_circles = np.where(y_circles == 0, -1, 1)



X_moons, y_moons = make_moons(n_samples=100, noise=0.3, random_state=0)

y_moons = np.where(y_moons == 0, -1, 1)


In [15]:
len(X_circles)

100

# Testing

## Hard Margin

In [16]:
y_train.shape

(8500,)

In [17]:
# separable case
SVM_model_Primal_hard_test = svm_train_primal_hard(X_class, y_class)

test_accyracy_Primal_Hard_test = svm_predict_primal(X_class,y_class, SVM_model_Primal_hard_test)

print("the accuracy of the Primal hard margin model is :", test_accyracy_Primal_Hard_test)
print("the bias term is ", SVM_model_Primal_hard_test[0], " the weights are ", SVM_model_Primal_hard_test[1:])

optimal
the accuracy of the Primal hard margin model is : 1.0
the bias term is  0.0038148465464497186  the weights are  [-1.75451608  0.0777831 ]


In [18]:
SVM_model_Dual_Hard_test = svm_train_dual_hard(X_class , y_class)
test_accuracy_Dual_Hard_test = svm_predict_dual(X_class , y_class , SVM_model_Dual_Hard_test)
print("the accuracy of the Dual hard margin model is :", test_accuracy_Dual_Hard_test)
print("the bias term is ", SVM_model_Dual_Hard_test[0], " the weights are ", SVM_model_Dual_Hard_test[1:])

optimal
the accuracy of the Dual hard margin model is : 1.0
the bias term is  0.003771064875461083  the weights are  [-1.7545558  0.0777976]


In [40]:
clf_hard_test = SVC(C = 1, kernel = 'linear', tol=0.0001,  max_iter=-1)
clf_hard_test.fit(X_class, y_class) 
print('w = ',clf_hard_test.coef_)
print('b = ',clf_hard_test.intercept_)
predicted_sklearn_hard_test = clf_hard_test.predict(X_class)

accuracy_score(y_class, predicted_sklearn_hard_test)

w =  [[-1.74054548  0.07716381]]
b =  [0.01177817]


1.0

## Soft Margin

In [20]:
SVM_model_Dual_Soft_test = svm_train_dual_soft(X_circles , y_circles,130)
test_accuracy_Dual_Soft_test = svm_predict_dual(X_circles , y_circles , SVM_model_Dual_Soft_test)
print("the accuracy of the Dual soft margin model is :", test_accuracy_Dual_Soft_test)
print("the bias term is ", SVM_model_Dual_Soft_test[0], " the weights are ", SVM_model_Dual_Soft_test[1:])

optimal
the accuracy of the Dual soft margin model is : 0.59
the bias term is  0.29015581075361685  the weights are  [ 0.7589323  -0.65157069]


In [21]:
SVM_model_Primal_Soft_test = svm_train_primal_soft(X_circles , y_circles,130)
test_accuracy_Primal_Soft_test = svm_predict_dual(X_circles , y_circles , SVM_model_Primal_Soft_test)
print("the accuracy of the Primal soft margin model is :", test_accuracy_Primal_Soft_test)
print("the bias term is ", SVM_model_Primal_Soft_test[0], " the weights are ", SVM_model_Primal_Soft_test[1:])

optimal
the accuracy of the Primal soft margin model is : 0.59
the bias term is  0.29023341561837973  the weights are  [ 0.7588472  -0.65150936]


In [22]:
clf_soft_test = SVC(C = 130, kernel = 'linear', tol=0.0001,  max_iter=-1)
clf_soft_test.fit(X_circles, y_circles) 
print('w = ',clf_soft_test.coef_)
print('b = ',clf_soft_test.intercept_)
predicted_sklearn_soft_test = clf_soft_test.predict(X_circles)

accuracy_score(y_circles, predicted_sklearn_soft_test)

w =  [[ 0.75886286 -0.65153027]]
b =  [0.29022996]


0.59

# Performance on Dataset

## Soft Margin

In [41]:
SVM_model_Dual_Soft_data = svm_train_dual_soft(x_train , y_train,1000)
test_accuracy_Dual_Soft_data_test_set = svm_predict_dual(x_test , y_test , SVM_model_Dual_Soft_data)
test_accuracy_Dual_Soft_data_train_set = svm_predict_dual(x_train , y_train , SVM_model_Dual_Soft_data)
print("the accuracy of the Dual soft margin model on testing set is :", test_accuracy_Dual_Soft_data_test_set)
print("the accuracy of the Dual soft margin model on training set is :", test_accuracy_Dual_Soft_data_train_set)

#print("the bias term is ", test_accuracy_Dual_Soft_data[0], " the weights are ", test_accuracy_Dual_Soft_data[1:])

optimal
the accuracy of the Dual soft margin model on testing set is : 0.968
the accuracy of the Dual soft margin model on training set is : 0.9774117647058823


In [42]:
SVM_model_Primal_Soft_data = svm_train_primal_soft(x_train , y_train,1000)
test_accuracy_Primal_Soft_data_testing_set = svm_predict_dual(x_test , y_test , SVM_model_Primal_Soft_data)
test_accuracy_Primal_Soft_data_training_set = svm_predict_dual(x_train , y_train , SVM_model_Primal_Soft_data)
print("the accuracy of the Primal soft margin model on testing set is:", test_accuracy_Primal_Soft_data_testing_set)
print("the accuracy of the Primal soft margin model on training set is:", test_accuracy_Primal_Soft_data_training_set)

#print("the bias term is ", test_accuracy_Primal_Soft_data[0], " the weights are ", test_accuracy_Primal_Soft_data[1:])

optimal
the accuracy of the Primal soft margin model on testing set is: 0.968
the accuracy of the Primal soft margin model on training set is: 0.9771764705882353


In [43]:
clf_soft_data = SVC(C = 1000, kernel = 'linear', tol=0.0001,  max_iter=-1)
clf_soft_data.fit(x_train, y_train) 
#print('w = ',clf_soft_data.coef_)
#print('b = ',clf_soft_data.intercept_)
predicted_sklearn_soft_data_test_set = clf_soft_data.predict(x_test)
predicted_sklearn_soft_data_trainig_set = clf_soft_data.predict(x_train)


accuracy_score(y_test, predicted_sklearn_soft_data_test_set)
accuracy_score(y_train, predicted_sklearn_soft_data_trainig_set )

0.9775294117647059

In [44]:
accuracy_score(y_test, predicted_sklearn_soft_data_test_set)


0.9673333333333334

In [45]:
accuracy_score(y_train, predicted_sklearn_soft_data_trainig_set )

0.9775294117647059

In [26]:
# non-separable case

# SVM_Model_Primal_Soft  = svm_train_primal_soft(x_train, y_train,100)

# test_accuracy_Primal_Soft = svm_predict_primal(x_test,y_test, SVM_Model_Primal_Soft)

# print("the accuracy of the Primal soft margin model is :", test_accuracy_Primal_Soft)

In [27]:
# print("the bias term is ", SVM_Model_Primal_Soft[0], " the weights are ", SVM_Model_Primal_Soft[1:])

In [28]:
# svm_model_p_s = svm_train_primal_soft(x_train , y_train ,1200)

# svm_model_p_h = svm_train_primal_hard(x_train , y_train)

# SVM_model_Dual_Soft = svm_train_dual_soft(x_train , y_train ,100)

# test_accuracy_primal_soft = svm_predict_primal(x_test , y_test , svm_model_p_s)

# test_accuracy_primal_hard = svm_predict_primal(x_test , y_test , svm_model_p_h)



# print(test_accuracy_primal_soft)
# print(test_accuracy_primal_hard)
# # print(classification_report(y_test, test_accuracy))

In [29]:
# test_accuracy_dual_soft = svm_predict_dual(x_test , y_test , SVM_model_Dual_Soft)
# print(test_accuracy_dual_soft)

In [30]:
# svm_model_d_s

In [31]:
# test_accuracy_Dual_Hard = svm_predict_dual(x_test , y_test , SVM_model_Dual_Hard)
# print(test_accuracy_Dual_Hard)

In [32]:
# from sklearn.svm import SVC


# clf = SVC(C = 1200, kernel = 'linear')
# clf.fit(x_train, y_train) 
# predicted_svm_sk = clf.predict(x_test)
# # print('w = ',clf.coef_)
# # print('b = ',clf.intercept_)
# # print('Indices of support vectors = ', clf.support_)
# # print('Support vectors = ', clf.support_vectors_)
# # print('Number of support vectors for each class = ', clf.n_support_)
# # print('Coefficients of the support vector in the decision function = ', np.abs(clf.dual_coef_))
# print(accuracy_score(y_test, predicted_svm_sk))